In [1]:
import syft as sy
from syft.core.node.common.node_service.tff.tff_messages import TFFMessageWithReply

import os, sys

import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff    
import io
from tensorflow_federated.python.learning.model_utils import ModelWeights 


# np.random.seed(0)

/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [104]:
domain = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into node... done!


In [105]:
domain.datasets


['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'get_protobuf_schema', 'icon', 'id', 'metadatas', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.dataset_manager.dataset_manager_messages.GetDatasetsResponse'>
['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module_

Idx,Name,Description,Assets,Id
[0],Little MedNIST,Small dataset for TFF testing,"[""images""] -> [""labels""] ->",56e947fe-1fb1-4ebc-be2a-caf980cc153b
[1],Little MedNIST private,Small PRIVATE dataset for TFF testing,"[""images""] -> [""labels""] ->",119583d7-2b92-43cc-8110-cbf3f6666122


In [106]:
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(64*64,)),
      tf.keras.layers.Dense(6, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])


In [84]:
def remote_train_model(model_fn, dataset_id, rounds, no_clients):
    tf.config.optimizer.set_experimental_options({'disable_meta_optimizer': True})

    model = model_fn()

    input_spec = (tf.TensorSpec(shape=(1,64*64), dtype=tf.int32, name=None), 
                tf.TensorSpec(shape=(1,1), dtype=tf.int32, name=None))

    functional_model = tff.learning.models.functional_model_from_keras(keras_model=model, 
                                                                    loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),
                                                                    input_spec=input_spec)
    print(functional_model.input_spec[0])
    tff.learning.models.save_functional_model(functional_model=functional_model, path='tmp')

    import shutil
    shutil.make_archive('archive', 'zip', 'tmp')
    
    params = {
    'rounds': rounds,
    'dataset_id': dataset_id,
    'no_clients': no_clients,
    }

    with open('archive.zip', 'rb') as f:
        model_bytes = f.read()
    # model_bytes = None

    # params = {}
    msg = TFFMessageWithReply(params=params, model_bytes=model_bytes)
    reply_msg = domain.send_immediate_msg_with_reply(msg)
    
    payload = eval(reply_msg.payload)
    
    memfile = io.BytesIO()
    memfile.write(payload['trainable'])
    memfile.seek(0)
    trainable = np.load(memfile, allow_pickle=True)
    
    memfile2 = io.BytesIO()
    memfile2.write(payload['non_trainable'])
    memfile2.seek(0)
    non_trainable = np.load(memfile2, allow_pickle=True)

    modelweights = ModelWeights(list(trainable), list(non_trainable))
    model = model_fn()
    modelweights.assign_weights_to(model)
    return model

In [79]:
domain.datasets[0].id

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'get_protobuf_schema', 'icon', 'id', 'metadatas', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.dataset_manager.dataset_manager_messages.GetDatasetsResponse'>


'56e947fe-1fb1-4ebc-be2a-caf980cc153b'

In [85]:
model_fn = create_keras_model
dataset_id = domain.datasets[0].id
rounds = 10
no_clients = 2

model = remote_train_model(model_fn, dataset_id, rounds, no_clients)

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'get_protobuf_schema', 'icon', 'id', 'metadatas', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.dataset_manager.dataset_manager_messages.GetDatasetsResponse'>
TensorSpec(shape=(1, 4096), dtype=tf.int32, name=None)


INFO:tensorflow:Assets written to: tmp/assets


INFO:tensorflow:Assets written to: tmp/assets


['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'get_protobuf_schema', 'icon', 'id', 'payload', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.tff.tff_messages.TFFReplyMessage'>


In [107]:
model

In [87]:
# download MedNIST.pkl
if not os.path.exists("./MedNIST.pkl"):
    os.system('curl -O "https://media.githubusercontent.com/media/shubham3121/datasets/main/MedNIST/MedNIST.pkl"')
else:
    print("MedNIST already downloaded")

MedNIST already downloaded


In [88]:
# Helper Methods

import os
import json
import pandas as pd
from PIL import Image
from enum import Enum
from collections import defaultdict
import numpy as np
from syft.core.adp.data_subject_list import DataSubjectList


def get_label_mapping():
    # the data uses the following mapping
    mapping = {
        "AbdomenCT": 0, 
        "BreastMRI": 1, 
        "CXR": 2, 
        "ChestCT": 3, 
        "Hand": 4, 
        "HeadCT": 5
    }
    return mapping

df = pd.read_pickle("./MedNIST.pkl")
mapping = get_label_mapping()

total_num = df.shape[0]
print("Columns:", df.columns)
print("Total Images:", total_num)
print("Label Mapping", mapping)

Columns: Index(['patient_id', 'image', 'label'], dtype='object')
Total Images: 58954
Label Mapping {'AbdomenCT': 0, 'BreastMRI': 1, 'CXR': 2, 'ChestCT': 3, 'Hand': 4, 'HeadCT': 5}


In [94]:
x = df['image'][10000]
x = tf.reshape(x, [-1,64*64])

In [99]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

Virtual devices cannot be modified after being initialized


In [108]:
model.predict(x)

InternalError: Graph execution error:

Detected at node 'sequential_7/dense_7/MatMul' defined at (most recent call last):
    File "/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 619, in start
      self.io_loop.start()
    File "/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File

In [102]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True